## This is a databricks notebook to create a delta table using a public dataset
#### Please use the "databricks_create_delta.dbc" file to upload to databricks

In [ ]:
import pyspark
from delta import configure_spark_with_delta_pip

In [ ]:
builder = (
    pyspark.sql.SparkSession.builder.appName("example-app-delta")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [ ]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
dbutils.fs.unmount(mount_point = "/mnt/flightdata")

In [ ]:


client_id = ''
client_secret = ''
tenant_id = ''
container_name = ''
datalake_name = ''

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": client_id,
       "fs.azure.account.oauth2.client.secret": client_secret,
       "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = f"abfss://{container_name}@{datalake_name}.dfs.core.windows.net/",
mount_point = "/mnt/flightdata",
extra_configs = configs)


In [ ]:
%sh
ls -l /dbfs/mnt/flightdata

In [ ]:
%sh
wget https://github.com/Azure-Samples/AzureStorageSnippets/raw/master/blobs/tutorials/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2016_1.zip -P /dbfs/mnt/flightdata

In [ ]:
%sh
mkdir "/dbfs/mnt/flightdata/Delta"
unzip /dbfs/mnt/flightdata/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2016_1.zip -d /dbfs/mnt/flightdata/
mv "/dbfs/mnt/flightdata/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2016_1.csv" /dbfs/mnt/flightdata/On_Time.csv
rm /dbfs/mnt/flightdata/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2016_1.zip
rm /dbfs/mnt/flightdata/readme.html

In [ ]:
  # Use the previously established DBFS mount point to read the data.
# create a data frame to read data.

flightDF = spark.read.format('csv').options(
    header='true', inferschema='true').load("/mnt/flightdata/*.csv")

# read the airline csv file and write the output to parquet format for easy query.
# flightDF.write.mode("append").parquet("/mnt/flightdata/parquet/flights")
print("Done")

In [ ]:
# flightDF.write.format("delta").saveAsTable("table1")
flightDF.write.format("delta").mode("overwrite").option('overwriteSchema','true').save("/mnt/flightdata/Delta")

In [ ]:
loaded_df = spark.read.format('delta').load('/mnt/flightdata/Delta')

In [ ]:
display(loaded_df)

In [ ]:
display(flightDF)

In [ ]:
flightDF.count()

In [ ]:
loaded_df.count()